## Set up the environment

In [1]:
import sagemaker as sage
from sagemaker import get_execution_role

MODEL_PACKAGE_ARN ="arn:aws:sagemaker:[region]:[your aws account id]:model-package/right-party-contact-likelihood-[XXXX]"

MODEL_NAME = "[Specify a name for your model e.g. last segment of Model Arn]"

# S3 paths for input and output data 
# Ensure the buckets are in the same region as subscribed model
S3_INPUT_PATH = "s3://[INPUT_BUCKET_NAME]/[INPUT_FILE]"
S3_OUTPUT_PATH = "s3://[OUTPUT_BUCKET_NAME]/output"
S3_OUTPUT_FILE = S3_OUTPUT_PATH + "/[INPUT_FILE].out"

# Specify one of the supported real time instances for the subscribed model
INFERENCE_INSTANCE = "ml.p2.xlarge"

# Specify one of the supported batch transform instances for the subscribed model
BATCH_INSTANCE = "ml.m5.large"

# ROLE = get_execution_role()
# or specify role arn
# ensure the role has access to S3 buckets specified above
ROLE = "arn:aws:iam::[your aws account]:role/[your sagemaker role name]"

sagemaker_session = sage.Session()


## Realtime Inference

In [2]:
from sagemaker import ModelPackage
model = ModelPackage(name=MODEL_NAME,
                     role=ROLE,
                     model_package_arn=MODEL_PACKAGE_ARN,
                     sagemaker_session=sagemaker_session)

In [3]:
endpoint_name=MODEL_NAME + "-edp"

In [ ]:
predictor = model.deploy(1, INFERENCE_INSTANCE, endpoint_name=endpoint_name)

In [ ]:
# connect to the endpoint
predictor = sage.Predictor(
    endpoint_name,
    sagemaker_session=sagemaker_session
)

In [ ]:
import s3fs, json
fs = s3fs.S3FileSystem()

with fs.open(S3_INPUT_PATH) as f:    
    docByteArr = f.read()
    response = predictor.predict(docByteArr)
    result=response.decode('utf-8')
    print(json.loads(result))

In [ ]:
# Delete endpoint
predictor.delete_endpoint()

## Batch Transform Job

In [ ]:
transformer = sage.transformer.Transformer(
    base_transform_job_name='batch-transform',
    model_name=MODEL_NAME,
    instance_count=1,
    instance_type=BATCH_INSTANCE,
    output_path=S3_OUTPUT_PATH,
    sagemaker_session=sagemaker_session
)


In [ ]:
transformer.transform(
    S3_INPUT_PATH
)

# wait until prediction job is completed
transformer.wait()


#### Inspect the Batch Transform Output in S3

In [ ]:
s3_client = sage.s3.S3Downloader
s3_client.read_file(S3_OUTPUT_FILE)